In [44]:
# List of imports
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import cv2 
import os
import pandas as pd

import tensorflow as tf
# from tensorflow.keras.preprocessing image import ImageDataGenerator, image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix

import itertools
import shutil
import random
%matplotlib inline

In [ ]:
# # Review:
# - import shutil
# - import itertools


In [45]:
train_path = 'data/train'
valid_path = 'data/valid'
test_path = 'data/test'

In [ ]:
print(tf.__version__)
# 2.3.0

In [ ]:
# legacy: ImageDataGenerator (deprecated*???*)
# my_syntax: *???* -- unsure; question later
# current: image_dataset_from_directory

In [ ]:
# # Documentation: https://blog.tensorflow.org/2020/07/whats-new-in-tensorflow-2-3.html
# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#   “datasets/cats_and_dogs”,
#   validation_split=0.2,
#   subset="training",
#   seed=0,
#   image_size=(img_height, img_width),
#   batch_size=32)

# #Previous:
# train_batches = tf.keras.preprocessing.image_dataset_from_directory(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(train_path, target_size=(224,224), batch_size=10
# valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(valid_path, target_size=(224,224), batch_size=10)

## Update: 2020-07-27
## Documentation: https://blog.tensorflow.org/2020/07/whats-new-in-tensorflow-2-3.html

In [ ]:
# Previous: batch_size=32
# train_batches = tf.keras.preprocessing.image_dataset_from_directory("data/train", validation_split=0.2, subset="training", seed=0, image_size=(224,224), batch_size=32)
# Found 16116 files belonging to 11 classes.
# Using 12893 files for training.

In [ ]:
# Current: batch_size=10
# train_batches = tf.keras.preprocessing.image_dataset_from_directory("data/train", validation_split=0.2, subset="training", seed=0, image_size=(224,224), batch_size=10)
# Found 16116 files belonging to 11 classes.
# Using 12893 files for training.

In [ ]:
# valid_batches = tf.keras.preprocessing.image_dataset_from_directory("data/valid", validation_split=0.8, subset="validation", seed=0, image_size=(224,224), batch_size=10)
# Found 330 files belonging to 11 classes.
# Using 66 files for validation.
# ValueError: If `subset` is set, `validation_split` must be set, and inversely.

# Found 330 files belonging to 11 classes.
# Using 264 files for validation.
# Question: validation_split=0.8 (inverse of validation_split=0.2 for subset="training" ???)

In [ ]:
# test_batches = tf.keras.preprocessing.image_dataset_from_directory("data/test", seed=0, image_size=(224,224), batch_size=10)
# Found 55 files belonging to 11 classes.

In [ ]:
# Documentation: https://keras.io/api/preprocessing/image/

In [46]:
# Tutorial: https://deeplizard.com/learn/video/FNqp4ZY0wDY
# resize any training data to have height of 224 and 224 (expected by MobileNet)
train_batches = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(train_path, target_size=(224,224), batch_size=10)
# Found 16116 images belonging to 11 classes.

Found 16116 images belonging to 11 classes.


In [47]:
valid_batches = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(valid_path, target_size=(224,224), batch_size=10)
# Found 330 images belonging to 11 classes.

Found 330 images belonging to 11 classes.


In [48]:
test_batches = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(test_path, target_size=(224,224), batch_size=10, shuffle=False)
# Found 55 images belonging to 11 classes.

Found 55 images belonging to 11 classes.


In [13]:
# train_batches.num_classes
# # 11

11

In [ ]:
# Review: "assert"
# Definition: assert -- The assert keyword is used when debugging code. The assert keyword lets you test if a condition in your code returns True, if not, the program will raise an AssertionError. You can write a message to be written if the code returns False, check the example below.

# Video: https://deeplizard.com/learn/video/LhEMXbjGV_4
# # Concept:
# - MobileNet
# - transfer learning

# Compare: tf.keras.preprocessing.image.ImageDataGenerator() VS. tf.keras.preprocessing.image_dataset_from_directory()

In [ ]:
# Tutorial:
# https://deeplizard.com/learn/video/FNqp4ZY0wDY

# Quiz:
# - After obtaining the data, the next step is to _______________.
# -- True

# - After obtaining the data, the next step is to _______________.
# -- Organize the data

# - Which function do we call to generate batches of training, validation, and test data?
# -- ImageDataGenerator.flow_from_directory()

# - Which parameter do we set in flow_from_directory() to resize images?
# -- target_size

# - What preprocessing_function do we use for the appropriate MobileNet preprocessing?
# -- tf.keras.applications.mobilenet.preprocess_input

In [49]:
# Assert number of images to what we expect
assert train_batches.n == 16116
assert valid_batches.n == 330
assert test_batches.n == 55
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 11

In [8]:
# type(train_batches)
# tensorflow.python.keras.preprocessing.image.DirectoryIterator

tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [7]:
# train_batches
# <tensorflow.python.keras.preprocessing.image.DirectoryIterator at 0x11d755640>

In [ ]:
# Tutorial: https://deeplizard.com/learn/video/FNqp4ZY0wDY
# Compare: MobileNet VS. vgg16
# Video

In [87]:
mobile = tf.keras.applications.mobilenet.MobileNet()
# Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/mobilenet/mobilenet_1_0_224_tf.h5
# 17227776/17225924 [==============================] - 3s 0us/step

In [88]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [89]:
# Store 6th to last layer as x (cats and dogs image classificaiton model skipped until 23rd layer)
# Build new model using MobileNet up to the 6th to last layer
# Last 5 layers are left OUT of the original MobileNet

# Ouput layer (predictions) contains 11 (10 digits from 0 to 9 plus "unknown") output nodes correspond to each of digit sign language classes
# Previous: predictions = Dense(2, activation='softmax')(x)


x = mobile.layers[-6].output
predictions = Dense(11, activation='softmax')(x)
model = Model(mobile.input, predictions)

# model = Model(input=mobile.input, predictions)
# SyntaxError: positional argument follows keyword argument
# model = Model(inputs=mobile.input, output=predictions)
# TypeError: ('Keyword argument not understood:', 'inputs')

# Debug: model = Model(mobile.input, predictions)

In [90]:
# Up to global_average_pooling2d_2 layer
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

In [ ]:
# main change is how many layers to train on new dataset
# keep most of MobileNet architecture  and ImageNet frozen layers

In [ ]:
## Tutorial: https://deeplizard.com/learn/video/4Tcqw5oIfIg
# Original MobileNet architecture has 88 layers
# use softmax activation function
# Model constructor
# instance of Model class
# 2 output layers

In [91]:
# Use up to the 23rd layers and below for new mdoel (sign language digits)
for layer in model.layers[:-23]:
    layer.trainable = False

In [62]:
# freeze weights of all layers except last 5 layers in our NEW (cats and dogs image classification)
# only last 5 layers of our model will be traveled
# conv_pw_13 (Conv2D) layer and below

# for layer in model.layers[:-5]:
#     layer.trainable = False

In [ ]:
# Error:
# model = Model(inputs=mobile.input, outputs=predictions)


In [ ]:
# Video: https://www.youtube.com/watch?v=FNqp4ZY0wDY&ab_channel=deeplizard
# dog and cats image classification VS. sign language
# 23rd to last layer; original ImageNet weights will stay in place
# for laye rin model.layers[:-23]:
#     layer.trainable=False

### Train the model

In [70]:
# model.output_shape
# (None, 2)

(None, 2)

In [67]:
# Debug: (Attempt)
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
# NameError: name 'y' is not defined
# Status: Not Working

NameError: name 'y' is not defined

In [92]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [93]:
# epochs = 60 for sign language digits (epoch = 30 for cat and dog model)
model.fit_generator(train_batches, steps_per_epoch=18, validation_data=valid_batches, validation_steps=3, epochs=60, verbose=2)

# Error:
# InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[10,2] labels_size=[10,11]
# 	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at <ipython-input-65-7a72a41b09c9>:2) ]] [Op:__inference_train_function_10533]

# Function call stack:
# train_function

# Debug: https://github.com/tensorflow/models/issues/6036
# Reason: resolved when numer of classes in train and test were different while using Image Generator
# Computational Speed: 6m 48.65s

Epoch 1/60
18/18 - 12s - loss: 2.3576 - accuracy: 0.2333 - val_loss: 2.5545 - val_accuracy: 0.3000
Epoch 2/60
18/18 - 11s - loss: 1.7628 - accuracy: 0.4556 - val_loss: 2.2662 - val_accuracy: 0.3333
Epoch 3/60
18/18 - 10s - loss: 1.3000 - accuracy: 0.6333 - val_loss: 2.1805 - val_accuracy: 0.4333
Epoch 4/60
18/18 - 6s - loss: 0.9227 - accuracy: 0.7500 - val_loss: 2.2748 - val_accuracy: 0.2333
Epoch 5/60
18/18 - 6s - loss: 0.8852 - accuracy: 0.7667 - val_loss: 1.0964 - val_accuracy: 0.5667
Epoch 6/60
18/18 - 6s - loss: 0.7340 - accuracy: 0.8111 - val_loss: 1.0250 - val_accuracy: 0.6000
Epoch 7/60
18/18 - 6s - loss: 0.6174 - accuracy: 0.8389 - val_loss: 1.0065 - val_accuracy: 0.6333
Epoch 8/60
18/18 - 7s - loss: 0.4685 - accuracy: 0.8722 - val_loss: 0.5929 - val_accuracy: 0.8333
Epoch 9/60
18/18 - 6s - loss: 0.4680 - accuracy: 0.8722 - val_loss: 0.5527 - val_accuracy: 0.8333
Epoch 10/60
18/18 - 6s - loss: 0.4569 - accuracy: 0.8722 - val_loss: 0.4638 - val_accuracy: 0.9000
Epoch 11/60
18/1

In [ ]:
# Rerun model to save time; change from 60 epochs to 30 epochs (when accuracy stabilizes to close to 100%) 
model.fit_generator(train_batches, steps_per_epoch=18, validation_data=valid_batches, validation_steps=3, epochs=30, verbose=2)

In [94]:
model.summary

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
json_file.write(model_json)
# serialize weights to HDF5
# weights define your model
model.save_weights("model.h5")
print("Saved model to disk")

In [96]:
# Documentation: https://www.tensorflow.org/tutorials/keras/save_and_load
!pip install -q pyyaml h5py  # Required to save models in HDF5 format

# Error: /bin/bash: /usr/local/bin/pip: /usr/local/opt/python/bin/python3.7: bad interpreter: No such file or directory

/bin/bash: /usr/local/bin/pip: /usr/local/opt/python/bin/python3.7: bad interpreter: No such file or directory


In [100]:
# !/tmpfs/src/tf_docs_env/bin/python -m pip install --upgrade pi
# Error: /bin/bash: /tmpfs/src/tf_docs_env/bin/python: No such file or directory

/bin/bash: /tmpfs/src/tf_docs_env/bin/python: No such file or directory


In [108]:
!which python
# /usr/local/bin/python

/usr/local/bin/python


In [1]:
!python --version
# Python 2.7.16

Python 2.7.16


In [111]:
conda env

Note: you may need to restart the kernel to use updated packages.


In [116]:
!conda create -n py37 python=3.7 anaconda

ERROR: The install method you used for conda--probably either `pip install conda`
or `easy_install conda`--is not compatible with using conda as an application.
If your intention is to install conda as a standalone application, currently
supported install methods include the Anaconda installer and the miniconda
installer.  You can download the miniconda installer from
https://conda.io/miniconda.html.



In [112]:
!conda activate tensorflow


# ERROR: The install method you used for conda--probably either `pip install conda`
# or `easy_install conda`--is not compatible with using conda as an application.
# If your intention is to install conda as a standalone application, currently
# supported install methods include the Anaconda installer and the miniconda
# installer.  You can download the miniconda installer from
# https://conda.io/miniconda.html.

ERROR: The install method you used for conda--probably either `pip install conda`
or `easy_install conda`--is not compatible with using conda as an application.
If your intention is to install conda as a standalone application, currently
supported install methods include the Anaconda installer and the miniconda
installer.  You can download the miniconda installer from
https://conda.io/miniconda.html.



In [ ]:
# fine tuned MobileNet model ready
# predicting new unseen images from our test set

In [ ]:
# %%capture cap --no-stderr
# print 'stuff'
# with open('output.txt', 'w') as f:
#     f.write(cap.stdout)

In [ ]:
# Review:
# - %%writefile
# - %%capture -- https://www.scrygroup.com/tutorial/2019-03-09/capturing_jupyter_cell_output/

In [40]:
# %%capture cap_out --no-stderr
# mobile.summary()

In [37]:
# with open('output.txt', 'w') as out:
#    out.write(cap.stdout)

In [41]:
# cap_out

In [42]:
# new_var = cap_out.stdout

In [ ]:
# new_var

In [ ]:
# File path of data and categories listed
DATADIR = 'data/'
CATEGORIES = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "unknown"]

In [ ]:
# Display an example image of each of the categories in grayscale
for category in CATEGORIES:
    path = os.path.join(DATADIR,category)  # create path to categories
    for img in os.listdir(path):
      # iterate over each image
      # convert to array
      img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  
      plt.imshow(img_array, cmap='gray')  # graph it
      plt.show()  # display!

      break  # we just want one for now so break
    break  #...and one more!

In [ ]:
# Because of the different varying image sizes, let's 

In [ ]:
IMG_SIZE = 64

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array, cmap='gray')
plt.show() # Show resize image

In [ ]:

def create_training_data():
    for category in CATEGORIES:  # do 0,1,2,3, .....

        path = os.path.join(DATADIR,category)  # create path to categories
        class_num = CATEGORIES.index(category)  # get the classification

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            
create_training_data() # call function

In [ ]:
import random
random.shuffle(training_data)

# Check
for sample in training_data[:10]:
    print(sample[1])

In [ ]:
X = []
Y = []

for features,label in training_data:
    X.append(features)
    Y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
import pickle

pickle_out = open("/working/X.pickle", "wb")               
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("/working/Y.pickle","wb")
pickle.dump(Y, pickle_out)
pickle_out.close()

In [ ]:
# - ImageDataGenerator
# - class_mode='input'

In [ ]:
# shear_range
# zoom_range
# horizontal_flip

In [ ]:
# generator shuffling

### Question
- resizing
- normalization / standardization
- TensorBoard evaluation